## Preliminary Analysis
These statements were taken from 18 months' worth of my chequing and credit card statements and aggregated into one document. The banking statements had sensitive information such as account numbers, etc. and so during the aggregation step they have been pre-screened by me.

Limitations of such an analysis include purchases from on general-purpose websites such as Amazon or other supermarkets, as well as paying back people via e-transfer. There could be multiple items purchased, as well as a variety of items offered by the marketplace, so the true nature of all purchases cannot be captured to be binned. 
Similarily, cash exchanges cannot be recorded and also skew both the amount of spending as well as groups of spending.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None




#### Preliminary Analysis 
Censoring, dropping columns without a deposit value, since we are only interested in the spending habits. 
Then general cleanup, dropping duplicates, conversion into datetime, sorting by datetime, and resetting the index.

In [10]:
def censor():
    data = pd.read_csv('CC&CA-Mar 2021-Sept 2021.csv')
    data.columns = ['date', 'recipient', 'withdraw', 'deposit', 'balance']
 
    dropped = data.drop_duplicates() # 282 rows now
    dropped['recipient'].nunique() # 173 unique values

    dropped['recipient'] = ['SAVINGS ACC' if x[0:6] == 'SSV TO' else x for x in dropped['recipient']]
    dropped['recipient'] = ['FIDO' if x[0:4] == 'FIDO' else x for x in dropped['recipient']]
    dropped['recipient'] = ['TD VISA' if x[0:7] == 'TD VISA' else x for x in dropped['recipient']]
    dropped['recipient'] = ['E-TRANSFER' if x[0:10] == 'SEND E-TFR' else x for x in dropped['recipient']]
    dropped['recipient'] = ['E-TRANSFER' if x[0:10] == 'E-TRANSFER' else x for x in dropped['recipient']]
    dropped['recipient'] = ['STEAM' if x[0:6] == 'STEAMG' else x for x in dropped['recipient']]
    dropped['recipient'] = ['MCGILL' if x[0:6] == 'MCGILL' else x for x in dropped['recipient']]
    dropped['recipient'] = ['AMZN Mktp CA' if x[0:9] == 'AMZN Mktp' else x for x in dropped['recipient']]
    dropped['recipient'] = ['AMZN Mktp CA' if x[0:9] == 'Amazon.ca' else x for x in dropped['recipient']]
    dropped['recipient'] = ['AMZN Mktp CA' if x[0:9] == 'AMZ*Amazo' else x for x in dropped['recipient']]
    dropped['recipient'] = ['FIZZ' if x[0:4] == 'FIZZ' else x for x in dropped['recipient']]
    dropped['recipient'] = ['FREEDOM MOBILE' if x[0:14] == 'FREEDOM MOBILE' else x for x in dropped['recipient']]
    dropped['recipient'] = ['UPS' if x[0:3] == 'UPS' else x for x in dropped['recipient']]
    dropped['recipient'] = ['eBay' if x[0:4] == 'eBay' else x for x in dropped['recipient']]
    dropped['recipient'] = ['WESTJET' if x[0:7] == 'WESTJET' else x for x in dropped['recipient']]
    dropped['recipient'] = ['Lenovo' if x[0:6] == 'Lenovo' else x for x in dropped['recipient']]
    dropped['recipient'] = ['TFR-TO C/C' if x[6:9] == 'TFR' else x for x in dropped['recipient']]

    dropped.to_csv('CensoredBankStatements.csv', index=False)
censor()

In [2]:
dropped = pd.read_csv('CensoredBankStatements.csv')

withdraw = dropped[dropped['withdraw'].notna()]

withdraw["date"] = pd.to_datetime(withdraw["date"])
withdraw.sort_values(by='date', inplace=True)
withdraw.reset_index(inplace=True, drop=True)
withdraw.drop(['deposit'], axis=1, inplace=True)
sum_calc = pd.to_numeric(withdraw['withdraw'], errors='coerce')
sum_calc.sum()

37000.909999999996

#### Binning 
Research and contextual analysis allowed me to synthesize my own categories for spending: 
- Food: Restaurants and groceries. 
- Investments: Any transfer to another account.
- Productivity: Technology and other home office items. 
- Materieal Desires: General spending for material desires.
- Necessities: Rent, utilities, phone and internet, and transportation.
- Education: Tuition, books, textbooks, and supplementary materials for university classes. 
- Recreation: Video games, events, movies, and other forms of entertainment.
- Health and Vanity: Prescribed drugs and treatments, supplements, clothing, and other vanity items.

In [5]:
withdraw["category"] = np.nan

withdraw.drop(withdraw[withdraw['recipient'] == 'TD VISA'].index, inplace=True)
# Manual Binning
def bin():
    withdraw.loc[withdraw['recipient'] == 'COCO TEA          _F', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'SAVINGS ACC', 'category'] = 'Investments'
    withdraw.loc[withdraw['recipient'] == 'STEAM', 'category'] = 'Recreation'
    withdraw.loc[withdraw['recipient'] == 'ALBERTA EDUCATI   _V', 'category'] = 'Education'
    withdraw.loc[withdraw['recipient'] == 'PAYPAL *RE  283.98_V', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'TFR-TO C/C', 'category'] = 'Investments'
    withdraw.loc[withdraw['recipient'] == 'Lenovo', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'MCGILL', 'category'] = 'Education'
    withdraw.loc[withdraw['recipient'] == 'LONDON DRUGS 40   _F', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'FREEDOM MOBILE', 'category'] = 'Necessities'
    withdraw.loc[withdraw['recipient'] == 'UPS', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'PAYPAL *FA    2.27_V', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'PP*HUMBLEB   14.24_V', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'PAYPAL *CH    7.76_V', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'PAYPAL *CHENZHONG', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'EXPLAIN EVERYTHING SALES,', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'PAYPAL *ORION L   _V', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'EPC*EPIC GAMES STORE', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'E-TRANSFER', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'aliexpress', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'TRUEDAN 6219 CENTRE ST.', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'Fanatical', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'CDN SHR INVEST   INV', 'category'] = 'Investments'
    withdraw.loc[withdraw['recipient'] == 'H&M CA#109 - DEERFOOT', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == '6A SNACK BAR', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'DOLLARAMA # 494', 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'LONDON DRUGS 40', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'DAIRY QUEEN #27231', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'PHO VAN VIETNAMESE NOODLE', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'XINGFUTANG HARVEST HILLS', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'KIM CHI HOUSE', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'PARK ADMISSIONS', 'category'] = 'Recreation'
    withdraw.loc[withdraw['recipient'] == 'eBay', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'ROYAL TEA HOUSE', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'DELL CANADA INC', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'LONDON DRUGS POSTAL OUTLE', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'BRIXTON NEIGHBOURHOOD PUB', 'category'] = 'Food'
    withdraw.at[15, 'category'] = 'Productivity'
    withdraw.at[47, 'category'] = 'Productivity'
    withdraw.at[74, 'category'] = 'Productivity'
    withdraw.at[75, 'category'] = 'Health and Vanity'
    withdraw.at[77, 'category'] = 'Productivity'
    withdraw.at[78, 'category'] = 'Material Desires'
    withdraw.loc[withdraw['recipient'] == 'ETSY', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'NEWEGG CANADA INC', 'category'] = 'Productivity'
    withdraw.at[83, 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'T&T SUPERMARKET #014', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'BOWLS KOREAN BBQ', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'WESTJET', 'category'] = 'Productivity'
    withdraw.at[88, 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'DOLLARAMA # 79', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'IKEA MONTREAL       ', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'SHUYI TEALICIOUS MCGILL', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'CARTE OPUS ETUDIANTS E', 'category'] = 'Necessities'
    withdraw.loc[withdraw['recipient'] == 'PP*HUMBLEBUNDL HUMBLEBUND', 'category'] = 'Recreation'
    withdraw.loc[withdraw['recipient'] == 'FIZZ', 'category'] = 'Necessities'
    withdraw.at[98, 'category'] = 'Productivity'
    withdraw.at[99, 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'FIDO', 'category'] = 'Necessities'
    withdraw.loc[withdraw['recipient'] == 'WAL-MART SUPERCENTER#3094', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'JEAN COUTU #071', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'WINNERS 340', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'UNIQLOMONTREAL EATON CENT', 'category'] = 'Health and Vanity'
    withdraw.at[109, 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'dhgate.com', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'RESTAURANT BOUSTAN CRESCE', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'PHARMAPRIX #1957', 'category'] = 'Necessities'
    withdraw.at[114, 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'GRAND & TOY A0', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'HOT DOG MOBILE    _F', 'category'] = 'Food'
    withdraw.at[119, 'category'] = 'Education'
    withdraw.at[120, 'category'] = 'Health and Vanity'
    withdraw.at[123, 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'BIXI MONTREAL', 'category'] = 'Necessities'
    withdraw.loc[withdraw['recipient'] == 'STEAM PURCHASE', 'category'] = 'Recreation'
    withdraw.at[128, 'category'] = 'Education'
    withdraw.loc[withdraw['recipient'] == 'STAPLES.CA', 'category'] = 'Productivity'
    withdraw.at[137, 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'GOOGLE*DEV47APPS', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'MATCHA ZANMAI', 'category'] = 'Food'
    withdraw.at[142, 'category'] = 'Health and Vanity'
    withdraw.at[144, 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'KITS Kits', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'STM LANGELIER DIN101', 'category'] = 'Necessities'
    withdraw.loc[withdraw['recipient'] == 'JEAN COUTU #152', 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'DELL CANADA INC (CAD-C', 'category'] = 'Productivity'
    withdraw.at[153, 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'CHATIME', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'BEST BUY.CA # 900', 'category'] = 'Investments'
    withdraw.loc[withdraw['recipient'] == 'Basha AV DU PARC', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'ROCKABERRY ST LEONARD', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'SEPHORA.CA', 'category'] = 'Health and Vanity'
    withdraw.at[170, 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'GOOGLE*JINPAI', 'category'] = 'Donation'
    withdraw.loc[withdraw['recipient'] == 'RESTAURANT DEVI', 'category'] = 'Food'
    withdraw.loc[withdraw['recipient'] == 'BOOKDEPOSITORY.COM', 'category'] = 'Education'
    withdraw.at[186, 'category'] = 'Health and Vanity'
    withdraw.at[188, 'category'] = 'Health and Vanity'
    withdraw.loc[withdraw['recipient'] == 'Memory Express', 'category'] = 'Productivity'
    withdraw.loc[withdraw['recipient'] == 'PAYPAL *BANGGOOD', 'category'] = 'Material Desires'

bin()
#This is called each time to help visualize what else is left to be binned.
empty = withdraw.loc[withdraw['category'].isna()]
empty

,date,recipient,withdraw,balance,category


In [4]:
withdraw.to_csv('./June2020-December2021BinnedSpending.csv')
withdraw['category'].value_counts()

Investments          43
Material Desires     31
Productivity         30
Food                 24
Health and Vanity    23
Necessities          20
Recreation            9
Education             8
Donation              1
Name: category, dtype: int64